# Drug Traffiking experiment: improved model

This notebooks contains the data processing, building and training part of the model used for the enhanced iteration of the drug traffiking experiment. 

In [8]:
import shap
import random
import witwidget
import numpy as np
import pandas as pd
from keras import layers
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.utils import class_weight
from sklearn.preprocessing import MinMaxScaler
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric
from sklearn.model_selection import train_test_split
from aif360.algorithms.preprocessing import Reweighing
from fairlearn.postprocessing import ThresholdOptimizer
from sklearn.metrics import classification_report,confusion_matrix
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

## Functions and Utils

In [ ]:
def minmax_scaler(data):
  scaler = MinMaxScaler()
  scaled = scaler.fit_transform(data)
  return scaled

def process_data(data):
  x = data.loc[:, data.columns != 'Tipo salida 2']
  y = data['Tipo salida 2']

  x_cat = x[['Region', 'Defensor', 'Desarrollo','extranjero']]
  x_cat['Region'] = label_encoder.fit_transform(x_cat['Region'])
  x_cat['Defensor'] = label_encoder.fit_transform(x_cat['Defensor'])
  x_cat['Desarrollo'] = label_encoder.fit_transform(x_cat['Desarrollo'])
  x_cat['extranjero'] = label_encoder.fit_transform(x_cat['extranjero'])

  x_num = x.loc[:, ~x.columns.isin(x_cat.columns)]

  x_norm = minmax_scaler(x_num)
  x_norm = pd.DataFrame(x_norm, columns = x_num.columns)

  x_norm.reset_index(drop=True, inplace=True)
  x_cat.reset_index(drop=True, inplace=True)

  x_fin = pd.concat([x_norm, x_cat], axis = 1)
  #y_fin = label_encoder.fit_transform(y)

  return x_fin

def custom_predict(examples_to_infer):

  preds = model1.predict(model_inputs)
  preds = [[1 - pred[0], pred[0]] for pred in preds]
  return preds

## Importing data and pre-processing

In [ ]:
path = r'traficoDrogas.csv'

data = pd.read_csv(path)

In [ ]:
## Cambiando las dos RM a una sola

data['Región (tribunal)']=data['Región (tribunal)'].replace('Metropolitana Sur','Metropolitana')
data['Región (tribunal)']=data['Región (tribunal)'].replace('Metropolitana Norte','Metropolitana')

## Sacar filas erróneas

data.drop(16690, inplace=True)
data.drop(16691, inplace=True)

## Agregar variable edad

data['Edad'] = np.nan

mu = 27 ## Edad promedio entre 18 y 34 años (concentran la mayoría de consumo de drogas) (estudio drogas senda 2020)
sigma = 8
random.seed(23)

for i in range(len(data)):
    data['Edad'][i] = round(max(18, min(np.random.normal(mu, sigma), 65)))

## Agregar variable extranjero

data['Extranjero'] = np.nan

for i in range(len(data)):
    if data['P.S. Expulsión'][i] == 'N':
        data['Extranjero'][i] = 'No'
    else:
        data['Extranjero'][i] = 'Sí'

In [ ]:
data.info()

In [ ]:
data.drop(data.columns[[2, 3, 4, 5, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]], axis=1, inplace=True)
data.rename(columns = {'Región (tribunal)':'Region', 'Grado desarrollo':'Desarrollo'}, inplace = True)

## Data Transformations

In [ ]:
x1 = data.loc[:, ~data.columns.isin(['Tipo salida 1', 'Tipo salida 2'])]
y1 = data.loc[:, data.columns.isin(['Tipo salida 1', 'Tipo salida 2'])]

In [ ]:
x_train1, x_test1, y_train1, y_test1 = train_test_split(x1, y1['Tipo salida 2'], test_size=0.3, random_state = 23)

## Del set de entrenamiento, se desprenden 1000 datos para generar un dataset de validación

x_val1 = x_train1[-1000:]
y_val1 = y_train1[-1000:]

In [ ]:
x_cat_train = x_train1[['Region', 'Defensor', 'Desarrollo','Extranjero']]
x_cat_test = x_test1[['Region', 'Defensor', 'Desarrollo','Extranjero']]
x_cat_val = x_val1[['Region', 'Defensor', 'Desarrollo','Extranjero']]

In [ ]:
label_encoder = preprocessing.LabelEncoder()

x_cat_train['Region'] = label_encoder.fit_transform(x_cat_train['Region'])
x_cat_train['Defensor'] = label_encoder.fit_transform(x_cat_train['Defensor'])
x_cat_train['Desarrollo'] = label_encoder.fit_transform(x_cat_train['Desarrollo'])
x_cat_train['Extranjero'] = label_encoder.fit_transform(x_cat_train['Extranjero'])

x_cat_test['Region'] = label_encoder.fit_transform(x_cat_test['Region'])
x_cat_test['Defensor'] = label_encoder.fit_transform(x_cat_test['Defensor'])
x_cat_test['Desarrollo'] = label_encoder.fit_transform(x_cat_test['Desarrollo'])
x_cat_test['Extranjero'] = label_encoder.fit_transform(x_cat_test['Extranjero'])

x_cat_val['Region'] = label_encoder.fit_transform(x_cat_val['Region'])
x_cat_val['Defensor'] = label_encoder.fit_transform(x_cat_val['Defensor'])
x_cat_val['Desarrollo'] = label_encoder.fit_transform(x_cat_val['Desarrollo'])
x_cat_val['Extranjero'] = label_encoder.fit_transform(x_cat_val['Extranjero'])

In [ ]:
x_num_train = x_train1.loc[:, ~x_train1.columns.isin(x_cat_train.columns)]
x_num_test = x_test1.loc[:, ~x_test1.columns.isin(x_cat_test.columns)]
x_num_val = x_val1.loc[:, ~x_val1.columns.isin(x_cat_val.columns)]

In [ ]:
x_norm_train = minmax_scaler(x_num_train)
x_norm_train = pd.DataFrame(x_norm_train, columns = x_num_train.columns)

x_norm_test = minmax_scaler(x_num_test)
x_norm_test = pd.DataFrame(x_norm_test, columns = x_num_test.columns)

x_norm_val = minmax_scaler(x_num_val)
x_norm_val = pd.DataFrame(x_norm_val, columns = x_num_val.columns)

In [ ]:
x_norm_train.reset_index(drop=True, inplace=True)
x_cat_train.reset_index(drop=True, inplace=True)

x_norm_test.reset_index(drop=True, inplace=True)
x_cat_test.reset_index(drop=True, inplace=True)

x_norm_val.reset_index(drop=True, inplace=True)
x_cat_val.reset_index(drop=True, inplace=True)

In [ ]:
x_train_fin = pd.concat([x_norm_train, x_cat_train], axis = 1)
x_test_fin = pd.concat([x_norm_test, x_cat_test], axis = 1)
x_val_fin = pd.concat([x_norm_val, x_cat_val], axis = 1)

y_train_fin = label_encoder.fit_transform(y_train1)
y_test_fin = label_encoder.fit_transform(y_test1)
y_val_fin = label_encoder.fit_transform(y_val1)

## Using AIF360

In [ ]:
dataset_train = BinaryLabelDataset(
    df = pd.concat([x_train_fin,pd.DataFrame(y_train_fin, columns = ['Tipo salida 2'])], axis = 1),
    label_names = ['Tipo salida 2'],
    protected_attribute_names = ['Region_alt'], #['Extranjero'],
    favorable_label = 1,
    unfavorable_label = 0
)

privileged_groups = [{'Region_alt': 1}] #[{'Extranjero': 1}]
unprivileged_groups = [{'Region_alt': 0}] #[{'Extranjero': 0}]

In [ ]:
metric_orig_train = BinaryLabelDatasetMetric(dataset_train,
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)


# Métrica de equidad (mean difference)
metric_f = metric_orig_train.mean_difference()
print("Diferencia en los resultados medios entre grupos privilegiados y no privilegiados = %f" % metric_f)

In [ ]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)

dataset_transf_train = RW.fit_transform(dataset_train)

In [ ]:
metric_orig_train = BinaryLabelDatasetMetric(dataset_transf_train,
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

# Métrica de equidad (mean difference)
metric_f_2 = metric_orig_train.mean_difference()

print("Diferencia en los resultados medios entre grupos privilegiados y no privilegiados = %f" % metric_f_2)

In [ ]:
x_train_fin.drop(columns = ['Region_alt'], inplace = True)
x_test_fin.drop(columns = ['Region_alt'], inplace = True)
x_val_fin.drop(columns = ['Region_alt'], inplace = True)

# x_train_fin.drop(columns = ['Extranjero'], inplace = True)
# x_test_fin.drop(columns = ['Extranjero'], inplace = True)
# x_val_fin.drop(columns = ['Extranjero'], inplace = True)

In [ ]:
train_fin =  pd.concat([x_train_fin, pd.DataFrame(y_train_fin, columns = ['Tipo salida 2'])], axis = 1)
train_fin

## Building the model

In [ ]:
num_classes1 = 1
num_features1 = x_train_fin.shape[1]
num_output1 = 1

num_layers_01 = 14
num_layers_11 = 12

epochs1 = 70
learning_rate1 = 0.01
batch_size1 = 1024

In [ ]:
inputs1 = keras.Input(shape = (num_features1,), name = "data")

x11 = layers.Dense(num_layers_01, activation = "relu", name = "dense_1")(inputs1)
x21 = layers.Dense(num_layers_11, activation = "relu", name = "dense_2")(x11)

outputs1 = layers.Dense(num_output1, activation = "sigmoid", name = "predictions")(x21)

model1 = keras.Model(inputs = inputs1, outputs = outputs1)

In [ ]:
class_weights = class_weight.compute_class_weight(class_weight = 'balanced', classes = np.unique(y_train_fin), y = y_train_fin)
class_weights = dict(enumerate(class_weights))

sample_weight = dataset_transf_train.instance_weights

## Training and metrics

In [ ]:
## Compilando el modelo

model1.compile(
    optimizer = keras.optimizers.Adam(learning_rate = learning_rate1),
    loss = keras.losses.BinaryCrossentropy(from_logits = False),
    metrics = [keras.metrics.Precision(name='precision_05', thresholds=0.5),
               keras.metrics.Precision(name='precision_025', thresholds=0.25),
               keras.metrics.Recall(name='recall_05', thresholds=0.5),
               keras.metrics.Recall(name='recall_025', thresholds=0.25),
               keras.metrics.FalseNegatives(name='fn_05', thresholds=0.5),
               keras.metrics.FalseNegatives(name='fn_025', thresholds=0.25),
               keras.metrics.BinaryAccuracy(name='accuracy_05', threshold=0.5),
               keras.metrics.BinaryAccuracy(name='accuracy_025', threshold=0.25)],
)

In [ ]:
# Entrenando el modelo

training1 = model1.fit(
    x_train_fin,
    y_train_fin,
    batch_size = batch_size1,
    epochs = epochs1,
    class_weight = class_weights,
    sample_weight = sample_weight,
    validation_data = (x_val_fin, y_val_fin)
)

## Model Evaluation

In [ ]:
## Generando predicciones

predict_train1 = (model1.predict(x_train_fin) > 0.5).astype(int)
predict_test1 = (model1.predict(x_test_fin) > 0.5).astype(int)

In [ ]:
## Matriz de confusión para dataset de entrenamiento 1

print(confusion_matrix(y_train_fin,predict_train1))
print(classification_report(y_train_fin,predict_train1))

In [ ]:
## Matriz de confusión para dataset de prueba 1

print(confusion_matrix(y_test_fin,predict_test1))
print(classification_report(y_test_fin,predict_test1))

## Using Fairlearn

In [ ]:
postprocess_est = ThresholdOptimizer(
    estimator=model1,
    constraints="false_negative_rate_parity",  # Optimize FPR and FNR simultaneously
    objective="accuracy_score",
    prefit=True,
    predict_method="predict",
)

In [ ]:
postprocess_est.fit(x_train_fin, y_train_fin, sensitive_features = x_train_fin['Region'])

In [ ]:
predict_train2 = postprocess_est.predict(x_train_fin, sensitive_features = x_train_fin['Region'])
predict_test2 = postprocess_est.predict(x_test_fin, sensitive_features = x_test_fin['Region'])

In [ ]:
## Matriz de confusión para dataset de entrenamiento 2

print(confusion_matrix(y_train_fin,predict_train2))
print(classification_report(y_train_fin,predict_train2))

In [ ]:
## Matriz de confusión para dataset de prueba 2

print(confusion_matrix(y_test_fin,predict_test2))
print(classification_report(y_test_fin,predict_test2))

## Saving results and model

In [ ]:
test_results = pd.DataFrame(predict_test1, columns=['score'])

x_test1_alt = x_test1.reset_index(drop=True)
y_test1_alt = y_test1.reset_index(drop=True)

test = pd.concat([x_test1_alt, test_results, pd.DataFrame(y_test_fin)], axis = 1)

path2 = r'preds_dt_improved.csv'
test.to_csv(path2)

In [ ]:
model1.save('dt_improved.keras')

## SHAP Values

In [ ]:
explainer = shap.Explainer(model1, x_train_fin.values[:])
shap_values = explainer(x_train_fin.values[:])
shap_values

In [ ]:
shap_values.feature_names = list(x_train_fin.columns)

In [ ]:
shap_df = pd.DataFrame(shap_values.values, columns=shap_values.feature_names)

# Calcular el valor absoluto y luego el promedio para cada característica
shap_abs_avg = shap_df.abs().mean()
shap_avg = shap_df.mean()
shap_max = shap_df.max()
shap_min= shap_df.min()

print('Media absoluta: ', '\n\n', shap_abs_avg)
print('----------------------------------')
print('Media: ', '\n\n', shap_avg)
print('----------------------------------')
print('Máximo: ', '\n\n', shap_max)
print('----------------------------------')
print('Mínimo: ', '\n\n', shap_min)

In [ ]:
fig, ax = plt.subplots()
shap.plots.beeswarm(shap_values)

In [ ]:
fig.savefig("dt_improved_shap.pdf", bbox_inches="tight", format="pdf")  # o "shap_plot.svg" para formato SVG

## What if Tool

In [ ]:
num_datapoints = 5007
tool_height_in_px = 750

examples_labels = pd.concat([x_test_fin.reset_index(drop=True), pd.DataFrame(y_test_fin, columns = ['Tipo salida 2']).reset_index(drop=True)], axis=1)
columns_not_for_model_input = [examples_labels.columns.get_loc('Tipo salida 2')]

examples_wit = examples_labels.values.tolist()
column_names = examples_labels.columns.tolist()

model_inputs = np.delete(np.array(examples_wit[:num_datapoints]), columns_not_for_model_input, axis=1)

def custom_predict_shap(examples_to_infer):

  preds = model1.predict(model_inputs)
  preds = [[1 - pred[0], pred[0]] for pred in preds]

  shap_output = explainer(model_inputs)
  attributions = []
  for single_shap_output in shap_output:  # iteramos sobre cada resultado de shap (cada ejemplo)
    attrs = {}
    for i, col in enumerate(x_train_fin.columns):
      attrs[col] = single_shap_output.values[i]  # utilizamos single_shap_output.values
    attributions.append(attrs)

  ret = {'predictions': preds, 'attributions': attributions}

  return ret

In [ ]:
# Setup the tool with the test examples and the trained classifier

config_builder = WitConfigBuilder(examples_wit[:num_datapoints],column_names).set_custom_predict_fn(custom_predict_shap).set_target_feature('Tipo salida 2')
WitWidget(config_builder, height=tool_height_in_px)